In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/10 20:32:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Compiling /home/eczech/repos/ot-scoring/notebooks/pathinit.sc

import $file.$          , sparkinit._

import $file.$         , pathinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

In [23]:
val input_dir = Paths.get(EXTRACT_DIR.resolve("evidence_raw.json").toString)
val df = ss.read.json(input_dir.toString)
df

json at cmd22.sc:2

9 / 9

input_dir: java.nio.file.Path = /home/eczech/data/ot/extract/evidence_raw.json
df: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]
res22_2: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]

In [3]:
df.filter($"target.id".contains("uniprot") && $"target.id".contains("-")).select("target.id").show(10, 100)

show at cmd2.sc:1

1 / 1

show at cmd2.sc:1

4 / 4

show at cmd2.sc:1

4 / 4

+---+
| id|
+---+
+---+



In [4]:
val upIds = df
    .filter($"target.id".contains("uniprot") && $"target.id".contains("/"))
    .select("target.id").dropDuplicates().limit(10).collect().map(_(0))
val upSamp = df.filter($"target.id".isin(upIds:_*)).groupBy("target.id").count()
upSamp.show(10, 100)

collect at cmd3.sc:3

9 / 9

collect at cmd3.sc:3

1 / 1

show at cmd3.sc:5

9 / 9

show at cmd3.sc:5

1 / 1

+-------------------------------------+-----+
|                                   id|count|
+-------------------------------------+-----+
|http://identifiers.org/uniprot/P23219| 5811|
|http://identifiers.org/uniprot/P35354| 6850|
|http://identifiers.org/uniprot/P10275|  870|
|http://identifiers.org/uniprot/P08588| 1810|
|http://identifiers.org/uniprot/P62942| 1489|
|http://identifiers.org/uniprot/P00915|  142|
|http://identifiers.org/uniprot/O60840| 1080|
|http://identifiers.org/uniprot/Q01668| 1072|
|http://identifiers.org/uniprot/Q13698| 1078|
|http://identifiers.org/uniprot/Q13936| 1073|
+-------------------------------------+-----+



upIds: Array[Any] = Array(
  "http://identifiers.org/uniprot/P23219",
  "http://identifiers.org/uniprot/P35354",
  "http://identifiers.org/uniprot/P10275",
  "http://identifiers.org/uniprot/P08588",
  "http://identifiers.org/uniprot/P62942",
  "http://identifiers.org/uniprot/P00915",
  "http://identifiers.org/uniprot/O60840",
  "http://identifiers.org/uniprot/Q01668",
  "http://identifiers.org/uniprot/Q13698",
  "http://identifiers.org/uniprot/Q13936"
)
upSamp: org.apache.spark.sql.package.DataFrame = [id: string, count: bigint]

### Gene Data

In [5]:
import java.nio.file.Paths
val path = EXTRACT_DIR.resolve("gene.json")
val dfg = ss.read.json(path.toString)
dfg

json at cmd4.sc:3

16 / 16

import java.nio.file.Paths

path: java.nio.file.Path = /home/eczech/data/ot/extract/gene.json
dfg: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]
res4_3: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]

In [12]:
// Show ensembl ids for UniProt ids
val dfgUpSamp = dfg.filter($"uniprot_id".isin(upIds.map(_.toString.split("/").last):_*))
dfgUpSamp.select("uniprot_id", "uniprot_accessions", "ensembl_gene_id").show(10, 25)

show at cmd11.sc:2

1 / 1

show at cmd11.sc:2

4 / 4

show at cmd11.sc:2

11 / 11

+----------+-------------------------+---------------+
|uniprot_id|       uniprot_accessions|ensembl_gene_id|
+----------+-------------------------+---------------+
|    P35354| [Q16876, A8K802, P35354]|ENSG00000073756|
|    P10275|[B1AKD7, A2RUN2, E7EVX...|ENSG00000169083|
|    Q01668|[Q9UDC3, Q01668, Q71UT...|ENSG00000157388|
|    Q13936|[Q14743, Q99025, Q1392...|ENSG00000151067|
|    Q13698|[A4IF51, Q12896, Q1369...|ENSG00000081248|
|    P62942|[P62942, Q4VC47, Q9H10...|ENSG00000088832|
|    P08588|[Q5T5Y4, P08588, Q9UKG...|ENSG00000043591|
|    P00915|                 [P00915]|ENSG00000133742|
|    P23219|[B4E2S5, Q3HY28, Q1512...|ENSG00000095303|
|    O60840|[Q9UHB1, O43901, F5CIQ...|ENSG00000102001|
+----------+-------------------------+---------------+



dfgUpSamp: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]

In [5]:
df.withColumn("id_equals", $"id" === $"ensembl_gene_id").groupBy("id_equals").count().show()

show at cmd4.sc:1

16 / 16

show at cmd4.sc:1

1 / 1

+---------+-----+
|id_equals|count|
+---------+-----+
|     true|60564|
+---------+-----+



In [6]:
Seq(
  ("id1", "uid1", Seq("x", "y")),
  ("id2", null, Seq()),
  ("id2", "uid2", Seq())
).toDF("eid", "uid", "uia")
.select($"eid", explode_outer(concat(array($"uid"), $"uia"))).show()

show at cmd5.sc:6

1 / 1

show at cmd5.sc:6

2 / 2

+---+----+
|eid| col|
+---+----+
|id1|uid1|
|id1|   x|
|id1|   y|
|id2|null|
|id2|uid2|
+---+----+



In [7]:
df.filter($"uniprot_id".isNull).count()

count at cmd6.sc:1

16 / 16

count at cmd6.sc:1

1 / 1

res6: Long = 0L

In [8]:
df.withColumn("isBlank", $"uniprot_id" === "").groupBy("isBlank").count().show()

show at cmd7.sc:1

16 / 16

show at cmd7.sc:1

1 / 1

+-------+-----+
|isBlank|count|
+-------+-----+
|   true|40475|
|  false|20089|
+-------+-----+



In [9]:
df.select("uniprot_id", "uniprot_accessions").show(10, 100)

show at cmd8.sc:1

1 / 1

+----------+----------------------------------------+
|uniprot_id|                      uniprot_accessions|
+----------+----------------------------------------+
|          |                                      []|
|    Q8N158|                        [Q8N158, A4D2A7]|
|    Q96RY5|[Q9P2C1, A8MZL1, Q96RY5, B1AJY1, Q8NDN1]|
|    B1AL46|                        [A6NHL0, B1AL46]|
|    P49116|        [B6ZGT8, A8K3H5, P49116, P55092]|
|          |                                      []|
|    O15263|                                [O15263]|
|    Q70SY1|                [Q6P454, Q70SY1, Q6ZMR6]|
|    Q8NGZ5|                [Q5JQT2, Q6IEZ0, Q8NGZ5]|
|    Q8NH54|                [Q8NH54, A6NN77, Q6IFF7]|
+----------+----------------------------------------+
only showing top 10 rows



In [10]:
val dfid = df
    .select($"ensembl_gene_id", explode_outer(
        concat(array($"uniprot_id"), $"uniprot_accessions")
    ).as("uniprot_id"))
    .withColumn("uniprot_id", when(trim($"uniprot_id") =!= "", $"uniprot_id").otherwise(lit(null: String)))
    .dropDuplicates()
dfid.show(10, 100)

show at cmd9.sc:7

16 / 16

show at cmd9.sc:7

1 / 1

+---------------+----------+
|ensembl_gene_id|uniprot_id|
+---------------+----------+
|ENSG00000259490|      null|
|ENSG00000213420|    Q8N158|
|ENSG00000213420|    A4D2A7|
|ENSG00000007545|    Q96RY5|
|ENSG00000007545|    Q9P2C1|
|ENSG00000007545|    A8MZL1|
|ENSG00000007545|    B1AJY1|
|ENSG00000007545|    Q8NDN1|
|ENSG00000228570|    B1AL46|
|ENSG00000228570|    A6NHL0|
+---------------+----------+
only showing top 10 rows



dfid: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ensembl_gene_id: string, uniprot_id: string]

In [11]:
dfid.count()

count at cmd10.sc:1

16 / 16

count at cmd10.sc:1

1 / 1

res10: Long = 129053L

In [20]:
dfid.agg(countDistinct("ensembl_gene_id")).show()

show at cmd19.sc:1

16 / 16

show at cmd19.sc:1

200 / 200

show at cmd19.sc:1

1 / 1

+-------------------------------+
|count(DISTINCT ensembl_gene_id)|
+-------------------------------+
|                          60564|
+-------------------------------+



In [21]:
df.agg(countDistinct("ensembl_gene_id")).show()

show at cmd20.sc:1

16 / 16

show at cmd20.sc:1

200 / 200

show at cmd20.sc:1

1 / 1

+-------------------------------+
|count(DISTINCT ensembl_gene_id)|
+-------------------------------+
|                          60564|
+-------------------------------+



## Resources

In [11]:
lazy val MAIN_RESOURCE_DIR = REPO_DIR.resolve(Paths.get("src", "main", "resources"))

MAIN_RESOURCE_DIR: java.nio.file.Path = [lazy]

In [15]:
val path = MAIN_RESOURCE_DIR.resolve("data/genes_with_non_reference_ensembl_ids_lkp.tsv")
val dfa = ss.read
    .format("csv")
    .option("sep", ",")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(path.toString)
dfa

load at cmd14.sc:7

1 / 1

load at cmd14.sc:7

2 / 2

path: java.nio.file.Path = /home/eczech/repos/ot-scoring/src/main/resources/data/genes_with_non_reference_ensembl_ids_lkp.tsv
dfa: org.apache.spark.sql.package.DataFrame = [gene_symbol: string, reference: string ... 1 more field]
res14_2: org.apache.spark.sql.package.DataFrame = [gene_symbol: string, reference: string ... 1 more field]

In [16]:
dfa.show(3)

show at cmd15.sc:1

1 / 1

+-----------+---------------+---------------+
|gene_symbol|      reference|      alternate|
+-----------+---------------+---------------+
|     ABCB11|ENSG00000073734|ENSG00000276582|
|       AGER|ENSG00000204305|ENSG00000237405|
|        AGK|ENSG00000006530|ENSG00000262327|
+-----------+---------------+---------------+
only showing top 3 rows



In [20]:
val dfjt = df.select(element_at(split($"target.id", "/"), -1).as("tid")).dropDuplicates()
val dfj = dfjt.join(dfa, $"tid" === $"alternate", "left")
dfj.show(3)

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd19.sc:3

9 / 9

show at cmd19.sc:3

1 / 1

+---------------+-----------+---------+---------+
|            tid|gene_symbol|reference|alternate|
+---------------+-----------+---------+---------+
|ENSG00000166148|       null|     null|     null|
|ENSG00000162613|       null|     null|     null|
|ENSG00000107679|       null|     null|     null|
+---------------+-----------+---------+---------+
only showing top 3 rows



dfjt: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [tid: string]
dfj: org.apache.spark.sql.package.DataFrame = [tid: string, gene_symbol: string ... 2 more fields]

In [22]:
// Show examle genes that join to alternate id
dfj.filter($"reference".isNotNull).show(10, 25)

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd21.sc:1

9 / 9

show at cmd21.sc:1

1 / 1

+---------------+-----------+---------------+---------------+
|            tid|gene_symbol|      reference|      alternate|
+---------------+-----------+---------------+---------------+
|ENSG00000183214|       MICA|ENSG00000204520|ENSG00000183214|
|ENSG00000223532|      HLA-B|ENSG00000234745|ENSG00000223532|
|ENSG00000227715|      HLA-A|ENSG00000206503|ENSG00000227715|
|ENSG00000231179|       MICB|ENSG00000204516|ENSG00000231179|
|ENSG00000235233|       MICA|ENSG00000204520|ENSG00000235233|
|ENSG00000206240|   HLA-DRB1|ENSG00000196126|ENSG00000206240|
|ENSG00000231823|   HLA-DQA2|ENSG00000237541|ENSG00000231823|
|ENSG00000225845|      BTNL2|ENSG00000204290|ENSG00000225845|
|ENSG00000276146|     LILRB2|ENSG00000131042|ENSG00000276146|
|ENSG00000277594|      DDX52|ENSG00000278053|ENSG00000277594|
+---------------+-----------+---------------+---------------+
only showing top 10 rows



In [19]:
// Show matches for only unique id matches in both sets
dfj.withColumn("m", $"gene_symbol".isNull).groupBy("m").count().show

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd18.sc:1

9 / 9

show at cmd18.sc:1

1 / 1

show at cmd18.sc:1

1 / 1

+-----+-----+
|    m|count|
+-----+-----+
| true|36932|
|false|   38|
+-----+-----+



In [21]:
// Show matches for ALL rows of evidence, not just unique ids
df.select(element_at(split($"target.id", "/"), -1).as("tid"))
    .join(dfa, $"tid" === $"alternate", "left")
    .withColumn("m", $"gene_symbol".isNull).groupBy("m").count().show

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd20.sc:3

9 / 9

show at cmd20.sc:3

1 / 1

+-----+-------+
|    m|  count|
+-----+-------+
| true|1730042|
|false|    526|
+-----+-------+

